In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from PIL import Image, ImageTk
from datetime import datetime
import csv
import sqlite3
import pandas as pd

# In-memory data store
bookings = []
booking_id_counter = 1

cars_data = [
    {"name": "Toyota Corolla", "price_per_day": 2000},
    {"name": "Honda Civic", "price_per_day": 2500},
    {"name": "Suzuki Cultus", "price_per_day": 1500},
    {"name": "BMW 3 Series", "price_per_day": 8000},
    {"name": "Mercedes E-Class", "price_per_day": 12000}
]

# ================== DATABASE FUNCTIONS ==================
def init_db():
    conn = sqlite3.connect("car_rentals.db")
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS bookings (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            phone TEXT,
            cnic TEXT,
            car TEXT,
            days INTEGER,
            total REAL,
            date TEXT
        )
    ''')
    conn.commit()
    conn.close()

def load_from_sqlite():
    global bookings, booking_id_counter
    conn = sqlite3.connect("car_rentals.db")
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM bookings")
        rows = cursor.fetchall()
        bookings = []
        for row in rows:
            bookings.append({
                "id": row[0],
                "name": row[1],
                "phone": row[2],
                "cnic": row[3],
                "car": row[4],
                "days": row[5],
                "total": row[6],
                "date": row[7]
            })
        if bookings:
            booking_id_counter = max(b["id"] for b in bookings) + 1
    except Exception as e:
        print("Error loading from SQLite:", e)
    finally:
        conn.close()

def save_to_sqlite():
    conn = sqlite3.connect("car_rentals.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM bookings")  # Clear old data
    for b in bookings:
        cursor.execute('''
            INSERT INTO bookings (id, name, phone, cnic, car, days, total, date)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (b["id"], b["name"], b["phone"], b["cnic"], b["car"], b["days"], b["total"], b["date"]))
    conn.commit()
    conn.close()

# ================== ANIMATION FUNCTIONS ==================
def fade_in(widget, alpha=0):
    if alpha < 1.0:
        alpha += 0.1
        widget.attributes('-alpha', alpha)
        widget.after(30, lambda: fade_in(widget, alpha))

def slide_in(widget, x=0, y=-100):
    if y < 0:
        y += 10
        widget.place(x=x, y=y)
        widget.after(30, lambda: slide_in(widget, x, y))

# ================== LOGIN WINDOW ==================
class LoginWindow(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("🔐 Login - Car Rental System")
        self.geometry("900x600")
        self.resizable(False, False)
        self.configure(bg="#f0f0f0")


        self.login_frame = ttk.Frame(self)
        self.login_frame.place(relx=0.5, rely=0.5, anchor="center")
        ttk.Label(self.login_frame, text="🔐 Login", font=("Segoe UI", 24, "bold")).pack(pady=20)

        self.username_var = tk.StringVar()
        self.password_var = tk.StringVar()

        ttk.Label(self.login_frame, text="Username").pack(anchor="w")
        ttk.Entry(self.login_frame, textvariable=self.username_var, width=30).pack(pady=5)

        ttk.Label(self.login_frame, text="Password").pack(anchor="w")
        ttk.Entry(self.login_frame, textvariable=self.password_var, show="*", width=30).pack(pady=5)

        ttk.Button(self.login_frame, text="Login", command=self.login).pack(pady=20)
        fade_in(self)

    def login(self):
        username = self.username_var.get()
        password = self.password_var.get()
        if username == "a" and password == "p":
            self.destroy()
            load_from_sqlite()
            app = CarRentalApp()
            app.mainloop()
        else:
            messagebox.showerror("Error", "Invalid credentials!")

# ================== MAIN APPLICATION ==================
class CarRentalApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("🚗 Car Rental Management System")
        self.geometry("1200x700")
        self.configure(bg="#f2f4f7")
        self.style_widgets()
        self.create_widgets()
        self.load_data()
        fade_in(self)

    def style_widgets(self):
        style = ttk.Style()
        style.theme_use("clam")
        style.configure("TButton", font=("Segoe UI", 10), padding=10, background="#3c8dbc", foreground="white")
        style.map("TButton", background=[('active', '#2b6cb0')], relief=[('pressed', 'sunken')])
        style.configure("Treeview.Heading", font=("Segoe UI", 12, "bold"), background="#3c8dbc", foreground="white")
        style.configure("Treeview", rowheight=28, font=("Segoe UI", 11))
        style.configure("Header.TLabel", font=("Segoe UI", 20, "bold"), background="#f2f4f7", foreground="#333")

    def create_widgets(self):
        ttk.Label(self, text="🚗 Car Rental Dashboard", style="Header.TLabel").pack(pady=20)
        btn_frame = ttk.Frame(self)
        btn_frame.pack(pady=10)

        ttk.Button(btn_frame, text="➕ Add Booking", command=self.open_add_window).grid(row=0, column=0, padx=5)
        ttk.Button(btn_frame, text="✏️ Edit Booking", command=self.edit_booking).grid(row=0, column=1, padx=5)
        ttk.Button(btn_frame, text="🗑️ Delete Booking", command=self.delete_booking).grid(row=0, column=2, padx=5)
        ttk.Button(btn_frame, text="🖨️ Export CSV", command=self.export_to_csv).grid(row=0, column=3, padx=5)
        ttk.Button(btn_frame, text="💾 Save to DB", command=save_to_sqlite).grid(row=0, column=4, padx=5)

        search_frame = ttk.Frame(self)
        search_frame.pack(pady=10, fill=tk.X, padx=20)
        self.search_var = tk.StringVar()
        self.search_var.trace("w", lambda *args: self.filter_table())
        ttk.Entry(search_frame, textvariable=self.search_var, width=40).pack(side=tk.RIGHT, padx=5)
        ttk.Label(search_frame, text="🔍 Search by Name/Phone/CNIC").pack(side=tk.RIGHT)

        columns = ("ID", "Name", "Phone", "CNIC", "Car", "Days", "Total", "Date")
        self.tree = ttk.Treeview(self, columns=columns, show='headings')
        for col in columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100 if col == "ID" or col == "Days" else 150)
        self.tree.pack(padx=20, pady=10, fill=tk.BOTH, expand=True)

    def load_data(self):
        for item in self.tree.get_children():
            self.tree.delete(item)
        for booking in bookings:
            self.tree.insert("", tk.END, values=(
                booking["id"],
                booking["name"],
                booking["phone"],
                booking["cnic"],
                booking["car"],
                booking["days"],
                f"Rs {booking['total']}",
                booking["date"]
            ))

    def filter_table(self):
        query = self.search_var.get().lower()
        filtered = [b for b in bookings if any(str(val).lower().find(query) != -1 for val in b.values())]
        for item in self.tree.get_children():
            self.tree.delete(item)
        for booking in filtered:
            self.tree.insert("", tk.END, values=(
                booking["id"],
                booking["name"],
                booking["phone"],
                booking["cnic"],
                booking["car"],
                booking["days"],
                f"Rs {booking['total']}",
                booking["date"]
            ))

    def open_add_window(self):
        BookingWindow(self, mode="add")

    def edit_booking(self):
        selected = self.tree.selection()
        if not selected:
            messagebox.showwarning("Select Booking", "Please select a booking to edit.")
            return
        item = self.tree.item(selected[0])
        booking_id = item['values'][0]
        BookingWindow(self, mode="edit", booking_id=booking_id)

    def delete_booking(self):
        selected = self.tree.selection()
        if not selected:
            messagebox.showwarning("Select Booking", "Please select a booking to delete.")
            return
        item = self.tree.item(selected[0])
        booking_id = item['values'][0]
        confirm = messagebox.askyesno("Delete", "Are you sure you want to delete this booking?")
        if confirm:
            global bookings
            bookings = [b for b in bookings if b["id"] != booking_id]
            self.load_data()

    def export_to_csv(self):
        filename = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
        if not filename:
            return
        with open(filename, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["ID", "Name", "Phone", "CNIC", "Car", "Days", "Total", "Date"])
            for b in bookings:
                writer.writerow([
                    b["id"],
                    b["name"],
                    b["phone"],
                    b["cnic"],
                    b["car"],
                    b["days"],
                    b["total"],
                    b["date"]
                ])
        messagebox.showinfo("Exported", "Bookings exported successfully!")

# ================== BOOKING WINDOW (ADD / EDIT) ==================
class BookingWindow(tk.Toplevel):
    def __init__(self, parent, mode, booking_id=None):
        super().__init__(parent)
        self.parent = parent
        self.mode = mode
        self.booking_id = booking_id
        self.title("➕ Add New Booking" if mode == "add" else "✏️ Edit Booking")
        self.geometry("500x600")
        self.configure(bg="#ffffff")
        self.resizable(False, False)
        self.transient(parent)
        self.grab_set()
        fade_in(self)

        self.name_var = tk.StringVar()
        self.phone_var = tk.StringVar()
        self.cnic_var = tk.StringVar()
        self.days_var = tk.StringVar(value="1")
        self.car_var = tk.StringVar(value=cars_data[0]["name"])

        if mode == "edit":
            for b in bookings:
                if b["id"] == booking_id:
                    self.name_var.set(b["name"])
                    self.phone_var.set(b["phone"])
                    self.cnic_var.set(b["cnic"])
                    self.car_var.set(b["car"])
                    self.days_var.set(str(b["days"]))
                    break

        self.create_widgets()

    def create_widgets(self):
        frame = ttk.Frame(self, padding=20)
        frame.pack(fill=tk.BOTH, expand=True)

        ttk.Label(frame, text="Client Name").grid(row=0, column=0, sticky="w", pady=5)
        ttk.Entry(frame, textvariable=self.name_var, width=30).grid(row=0, column=1, pady=5)

        ttk.Label(frame, text="Phone (11 Digits)").grid(row=1, column=0, sticky="w", pady=5)
        ttk.Entry(frame, textvariable=self.phone_var, width=30).grid(row=1, column=1, pady=5)

        ttk.Label(frame, text="CNIC (13 Digits)").grid(row=2, column=0, sticky="w", pady=5)
        ttk.Entry(frame, textvariable=self.cnic_var, width=30).grid(row=2, column=1, pady=5)

        ttk.Label(frame, text="Select Car").grid(row=3, column=0, sticky="w", pady=5)
        car_names = [car["name"] for car in cars_data]
        ttk.OptionMenu(frame, self.car_var, *car_names).grid(row=3, column=1, sticky="ew", pady=5)

        ttk.Label(frame, text="Rent Days").grid(row=4, column=0, sticky="w", pady=5)
        ttk.Entry(frame, textvariable=self.days_var, width=30).grid(row=4, column=1, pady=5)

        ttk.Button(frame, text="🧮 Calculate Amount", command=self.calculate_amount).grid(row=5, column=0, columnspan=2, pady=10)
        self.amount_label = ttk.Label(frame, text="", foreground="green", font=("Segoe UI", 12, "bold"))
        self.amount_label.grid(row=6, column=0, columnspan=2, pady=5)

        ttk.Button(frame, text="✅ Confirm Booking", command=self.save_booking).grid(row=7, column=0, columnspan=2, pady=10)

    def calculate_amount(self):
        try:
            days = int(self.days_var.get())
            car_name = self.car_var.get()
            price = next(car["price_per_day"] for car in cars_data if car["name"] == car_name)
            total = price * days
            self.amount_label.config(text=f"Total Amount: Rs {total}")
        except:
            messagebox.showerror("Error", "Please enter valid days and select a car.")

    def save_booking(self):
        global booking_id_counter  # ✅ Global declaration at the top
        name = self.name_var.get().strip()
        phone = self.phone_var.get().strip()
        cnic = self.cnic_var.get().strip()
        days = self.days_var.get().strip()
    
        if not name:
            messagebox.showerror("Error", "Client name is required.")
            return
        if len(phone) != 11 or not phone.isdigit():
            messagebox.showerror("Error", "Phone must be exactly 11 digits.")
            return
        if len(cnic) != 13 or not cnic.isdigit():
            messagebox.showerror("Error", "CNIC must be exactly 13 digits.")
            return
        if not days.isdigit() or int(days) <= 0:
            messagebox.showerror("Error", "Enter valid number of days.")
            return
    
        car_name = self.car_var.get()
        price = next(car["price_per_day"] for car in cars_data if car["name"] == car_name)
        total = price * int(days)
        data = {
            "id": booking_id_counter if self.mode == "add" else self.booking_id,
            "name": name,
            "phone": phone,
            "cnic": cnic,
            "car": car_name,
            "days": int(days),
            "total": total,
            "date": datetime.now().strftime("%Y-%m-%d %H:%M")
        }
    
        if self.mode == "add":
            bookings.append(data)
            booking_id_counter += 1
        elif self.mode == "edit":
            for i, b in enumerate(bookings):
                if b["id"] == self.booking_id:
                    bookings[i] = data
                    break
    
        self.parent.load_data()
        messagebox.showinfo("Success", "Booking saved successfully!")
        ReceiptWindow(self.parent, data)
        fade_out(self)

# ================== RECEIPT WINDOW ==================
class ReceiptWindow(tk.Toplevel):
    def __init__(self, parent, data):
        super().__init__(parent)
        self.title("🧾 Booking Receipt")
        self.geometry("400x400")
        self.configure(bg="#ffffff")
        self.resizable(False, False)
        self.transient(parent)
        self.grab_set()
        fade_in(self)

        frame = ttk.Frame(self, padding=20)
        frame.pack(fill=tk.BOTH, expand=True)

        ttk.Label(frame, text="🚗 Car Rental Receipt", font=("Segoe UI", 16, "bold")).pack(pady=10)
        ttk.Label(frame, text=f"Name: {data['name']}").pack(anchor="w")
        ttk.Label(frame, text=f"Phone: {data['phone']}").pack(anchor="w")
        ttk.Label(frame, text=f"CNIC: {data['cnic']}").pack(anchor="w")
        ttk.Label(frame, text=f"Car: {data['car']}").pack(anchor="w")
        ttk.Label(frame, text=f"Days: {data['days']}").pack(anchor="w")
        ttk.Label(frame, text=f"Total: Rs {data['total']}").pack(anchor="w", pady=(10, 0))
        ttk.Label(frame, text=f"Date: {data['date']}").pack(anchor="w", pady=(5, 20))
        ttk.Button(frame, text="🖨️ Print Receipt", command=self.print_receipt).pack()

    def print_receipt(self):
        messagebox.showinfo("Print", "Receipt has been sent to printer.")

# ================== RUN THE APP ==================
if __name__ == "__main__":
    init_db()
    login = LoginWindow()
    login.mainloop()